# Computation of the session features

<ins>Aim</ins> - Compute the features ("request interarrival time", "session interarrival time" and "number of requests per session") needed for the data analysis by preprocessing the access log data.

<ins>Input</ins> - The structured logs as produced by the `Structuration.ipynb` notebook.

<ins>Output</ins> - Two CSV tables written in `./requests.csv.d` and `./sessions.csv.d`. They contain the features and addtional information.

<ins>Method</ins> - Read the strutured logs, compute the difference between arrival times of requests for each hosts, identify the sessions and compute the difference between arrival times of sessions.

<ins>Tools</ins> - SparkSQL from Spark 2.4.3, pandas 0.24.2

**Table of content**
* [Initialisation](#init)
* [Read the logs](#read)
* [Compute the "request interarrival time" feature](#request)
* [Compute the "session interarrival time" and "number of requests per session" features](#session)
* [Write on disk](#write)

## Initialisation <a class="anchor" id="init"></a>
Import libraries and start a Spark session.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd

In [2]:
spark = SparkSession.builder.config('spark.driver.memory','8G').getOrCreate()
spark

## Read the logs <a class="anchor" id="read"></a>
Read the structured logs produced by the `Stucturation.ipynb` notebook.

In [3]:
input_directory = './access.csv.d'

keys_type = [( 'remote_host',               StringType() ),
             ( 'request_first_line',        StringType() ),
             ( 'request_header_referer',    StringType() ),
             ( 'request_header_user_agent', StringType() ),
             ( 'request_method',            StringType() ),
             ( 'response_bytes_clf',        LongType()   ),
             ( 'status',                    StringType() ),
             ( 'timestamp_utc',             LongType()   )
            ]
schema = StructType([StructField(key, ktype, True) for key,ktype in keys_type])

access = spark.read.option('header','true').schema(schema).csv(input_directory)

Add a column identifying the client (IP + user agent).

In [4]:
access = access.withColumn('client',
                           concat(col('remote_host'),col('request_header_user_agent')))

Print basic properties of the log dataframe. <br>
Print a preview of the data.

In [5]:
print(f'Number of log entries: {access.count()}\n')
print('Schema and first rows:')
access.printSchema()
access.toPandas().sample(n=10).head(10)

Number of log entries: 5437038

Schema and first rows:
root
 |-- remote_host: string (nullable = true)
 |-- request_first_line: string (nullable = true)
 |-- request_header_referer: string (nullable = true)
 |-- request_header_user_agent: string (nullable = true)
 |-- request_method: string (nullable = true)
 |-- response_bytes_clf: long (nullable = true)
 |-- status: string (nullable = true)
 |-- timestamp_utc: long (nullable = true)
 |-- client: string (nullable = true)



,remote_host,request_first_line,request_header_referer,request_header_user_agent,request_method,response_bytes_clf,status,timestamp_utc,client
3426025,149.56.83.40,POST /administrator/index.php HTTP/1.1,-,-,POST,4498.0,200,1.486545e+09,149.56.83.40-
725044,31.184.253.29,POST /index.php?option=com_contact&view=contac...,-,-,POST,5.0,303,1.582289e+09,31.184.253.29-
4007276,158.69.5.181,POST /administrator/index.php HTTP/1.1,-,-,POST,4498.0,200,1.522889e+09,158.69.5.181-
1168665,42.236.10.106,GET /templates/jp_hotel/css/menu.css HTTP/1.1,http://www.almhuette-raith.at/,Mozilla/5.0 (Linux; U; Android 8.1.0; zh-CN; E...,GET,1457.0,200,1.563179e+09,42.236.10.106Mozilla/5.0 (Linux; U; Android 8....
1779363,5.113.35.73,GET /apache-log/access.log HTTP/1.1,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,GET,43192.0,206,1.527492e+09,5.113.35.73Mozilla/5.0 (Windows NT 6.1; Triden...
3936233,158.69.5.181,POST /administrator/index.php HTTP/1.1,-,-,POST,4498.0,200,1.522866e+09,158.69.5.181-
1109149,193.106.31.130,POST /administrator/index.php HTTP/1.0,-,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,POST,4481.0,200,1.560765e+09,193.106.31.130Mozilla/4.0 (compatible; MSIE 7....
5296168,13.84.43.203,POST //administrator/index.php HTTP/1.1,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...,POST,4501.0,200,1.572988e+09,13.84.43.203Mozilla/5.0 (Windows NT 10.0; Win6...
245153,52.171.38.247,GET //administrator/index.php HTTP/1.1,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...,GET,4270.0,200,1.573759e+09,52.171.38.247Mozilla/5.0 (Windows NT 10.0; Win...
3415473,149.56.83.40,POST /administrator/index.php HTTP/1.1,-,-,POST,4498.0,200,1.486541e+09,149.56.83.40-


## Compute the "request interarrival time" feature <a class="anchor" id="request"></a>
Define windows that will be used to apply lag function and cumulative sum function.

In [6]:
host_window = Window.partitionBy("client").orderBy("timestamp_utc")
host_window_cum = host_window.rowsBetween(Window.unboundedPreceding, 0)

Select only the host names and the timestamp from the logs.<br>
Add a column containing the arrival time of the previous request of the host (`null` if the host is seen for the first time in the log)

In [7]:
requests = access.select( ['client', 'timestamp_utc'] ) \
                 .withColumn('previous_request_timestamp_utc',
                             lag(col('timestamp_utc'))
                             .over(host_window) )

Add a column containing the **request interarrival time feature** and drop the column containing the arrival time of the previous request of the host.<br>
Set the request interarrival time to `null` if it is greater than the maximum session time.

In [8]:
max_session_time_seconds = 30*60

requests = requests.withColumn('request_interarrival_time',
                               col('timestamp_utc') - col('previous_request_timestamp_utc')) \
                   .select( ['client', 'timestamp_utc', 'request_interarrival_time'] )

requests = requests.withColumn('request_interarrival_time',
                               when(requests['request_interarrival_time'] > max_session_time_seconds, None)
                               .otherwise(requests['request_interarrival_time']) )

Add a column containing a boolean answering the question "Is it the first request of the session?".

In [9]:
requests = requests.withColumn('new_session',
                               col('request_interarrival_time').isNull() )

Add a column containing the number of the session to which the request belongs to (count from beginning of the log file).

In [10]:
requests = requests.withColumn('session_number',
                               sum(col('new_session').cast('integer'))
                               .over(host_window_cum) )

Add a column containing a boolean answering the question "Is it the first time the host appears in the logs?".

In [11]:
requests = requests.withColumn('first_connection',
                               col('new_session') & (col('session_number') == 1) )

Reorder the columns and print a random sample of the table.

In [12]:
%%time
requests = requests.select( ['client', 'timestamp_utc', 'first_connection',
                             'new_session','session_number','request_interarrival_time'] )
requests.toPandas().sample(n=10).head(10)

CPU times: user 26.4 s, sys: 1.96 s, total: 28.3 s
Wall time: 50.9 s


,client,timestamp_utc,first_connection,new_session,session_number,request_interarrival_time
4021075,13.84.43.203Mozilla/5.0 (Windows NT 10.0; Win6...,1572924904,False,False,1,1.0
3791045,205.167.170.15Go-http-client/1.1,1456870009,False,False,23,0.0
2829586,95.216.96.254Mozilla/5.0 (compatible; SEOkicks...,1570868552,False,False,21,1.0
3746136,5.112.66.178Mozilla/5.0 (Windows NT 6.1; Tride...,1529585517,False,False,1,0.0
2937983,42.126.25.173Mozilla/5.0 (Macintosh; Intel Mac...,1563083691,False,False,1,0.0
4394835,13.84.43.203Mozilla/5.0 (Windows NT 10.0; Win6...,1573065200,False,False,1,0.0
4933139,5.113.35.73Mozilla/5.0 (Windows NT 6.1; Triden...,1527494182,False,False,1,1.0
1270126,198.50.156.189-,1491042324,False,False,1,1.0
3080129,100.1.14.108python-requests/2.22.0,1569628489,False,False,15,0.0
4436578,138.197.111.212Mozilla/5.0 (X11; Datanyze; Lin...,1525405519,False,False,1,0.0


## Compute the "session interarrival time" and "number of requests per session" features <a class="anchor" id="session"></a>
Group the requests by host and session number to get the sessions arrival time and the **number of requests per session**.

In [13]:
sessions = requests.groupBy( ['client','session_number'] ) \
                   .agg( {'timestamp_utc':'min', '*':'count'} )
sessions = sessions.withColumnRenamed('min(timestamp_utc)', 'timestamp_utc') \
                   .withColumnRenamed('count(1)', 'number_of_requests')

Add a column containing the arrival time of the previous session of the host (`null` if it is the first session of the host in the logs)

In [14]:
sessions = sessions.withColumn('previous_session_timestamp_utc',
                               lag(col('timestamp_utc'))
                               .over(host_window) )

Add a column containing the **session interarrival time** (`null` if it is the first session of the host in the logs).

In [15]:
sessions = sessions.withColumn('session_interarrival_time',
                               col('timestamp_utc') - col('previous_session_timestamp_utc') )

Add a column containing the answer to the question "Is it the first session of the host in the logs?".

In [16]:
sessions = sessions.withColumn('first_session',
                               col('session_number') == 1 )

Select the column to keep in the session table and print a random sample.

In [17]:
%%time
sessions = sessions.select( ['client', 'timestamp_utc', 'first_session', 'session_number',
                             'number_of_requests', 'session_interarrival_time'] )
sessions.toPandas().sample(n=10).head(10)

CPU times: user 1.14 s, sys: 102 ms, total: 1.24 s
Wall time: 16.5 s


,client,timestamp_utc,first_session,session_number,number_of_requests,session_interarrival_time
231752,178.159.37.81Mozilla/5.0 (Windows NT 10.0; WOW...,1497712331,False,412,1,19722.0
192797,196.18.13.146Mozilla/5.0 (Windows NT 6.1; WOW6...,1554033776,False,4,1,19885.0
235059,184.175.214.54Mozilla/5.0 (Windows NT 6.1; Win...,1499271756,True,1,1,NaN
269229,178.159.37.81Mozilla/5.0 (Windows NT 6.3; WOW6...,1500772588,False,9,1,308577.0
112244,195.154.216.79Mozilla/5.0 (Windows NT 6.1; WOW...,1474497995,False,159,1,39463.0
147903,131.0.216.75Mozilla/5.0 (Windows NT 10.0; WOW6...,1585677333,True,1,5,NaN
239262,178.159.37.81Mozilla/5.0 (Windows NT 6.1; WOW6...,1494894781,False,269,1,5879.0
71997,49.205.77.48Mozilla/5.0 (Windows NT 10.0; Win6...,1582415233,True,1,1,NaN
52145,91.121.185.43Mozilla/5.0 (Windows NT 10.0; Win...,1589362516,False,447,1,29727.0
37828,73.209.138.183Mozilla/5.0 (Macintosh; Intel Ma...,1552078118,True,1,1,NaN


## Write on disk <a class="anchor" id="write"></a>
Write tables to CSV files.

In [18]:
%%time
requests.write.option('header','true').csv('./requests.csv.d', mode='overwrite')
sessions.write.option('header','true').csv('./sessions.csv.d', mode='overwrite')

CPU times: user 6.97 ms, sys: 3.94 ms, total: 10.9 ms
Wall time: 34.4 s


Close the Spark session.

In [19]:
spark.stop()

*(end of the ComputeFeatures notebook)*